In [0]:
!pip3 install ftfy
!pip3 install unidecode

In [0]:
#Definició de constants, variables globals i funcions auxiliars per extreure dades del portal de pisos buscocasa.ad
from bs4 import BeautifulSoup
from requests import get
from ftfy import fix_encoding 
import time
from random import randint
import unidecode
import datetime
import pandas as pd

def getPreu(oferta):   
  """
  Retorna el preu a partir del tag d'oferta
  :param oferta: bs4.element.tag. Contingut de la oferta
  :retorna: string el preu
  """
  preu = oferta.select('div.uk-float-right')[0].text.strip().split()[0]
  if preu == 'Consultar':
      preu = 'NaN'
  return preu.replace('.','')

def getNumVisites(oferta):
  """
  Retorna el número de visites a partir del tag d'oferta
  :param oferta: bs4.element.tag. Contingut de la oferta
  :retorna: string representant el nombre de visites
  """
  if len(oferta.select('div.uk-float-right'))==2:
      if oferta.select('div.uk-float-right')[1].text.strip().split()[-2] == "":
          numVisites = 'NaN'
      else:
          numVisites = oferta.select('div.uk-float-right')[1].text.strip().split()[-2]
      return numVisites.replace('.','')

def getTipusOferta(oferta):
  """
  Retorna el tipus d'oferta a partir del tag d'oferta
  :param oferta: bs4.element.tag. Contingut de la oferta
  :retorna: string indicant el tipus d'oferta
  """
  if oferta.find('em') != None:
      tipus = oferta.find('em').text.strip().split()[0]
      return tipus

def getNumHabitacions(oferta):
    """
    Retorna el nombre d'habitacions a partir del tag d'oferta
    :param oferta: bs4.element.tag. Contingut de la oferta 
    :retorna: string indicant el nombre d'habitacions

    """
    if oferta.find('em')!= None:
        numHab = oferta.find('em').text.strip().split()[-2]
        return numHab

def getM2(oferta):
  """
  Retorna els metres quadrats a partir del tag d'oferta
  :param oferta: bs4.element.tag. Contingut de la oferta  
  :retorna: string indicant els m2 del pis
  """
  if oferta.find('em')!= None:
      if oferta.find('em').text.strip().split()[2] != 'm2':
          metres = 'NaN'
      else:
          metres = oferta.find('em').text.strip().split()[1]
      return metres.replace('.','')
        
def getData(oferta):
    """
    Retorna la data de creació de l'oferta a partir del tag d'oferta
    :param oferta: bs4.element.tag. Contingut de la oferta  
    :retorna: string indicant la data de creació de la oferta
    """
    if len(oferta.select('div.uk-float-right'))==2:
        var = oferta.select('div.uk-float-right')[1].text.strip().split()
        if var[0] == '+':
            dtOferta = ' '.join(var[0:4])
        else:
            dtOferta = var[0]
        return dtOferta
            
def getParroquia(oferta, params):
    
    """
    Retorna la parròquia a partir del tag d'oferta
    :param oferta: bs4.element.tag. Contingut de la oferta  
    :retorna: string indicant el nom de la parròquia      
    """        
    parroquia = 'NaN'
    textTitol = unidecode.unidecode(oferta.select(params['tag_titol'])[0].text.strip().lower().replace("."," ").replace("-"," ").replace(","," "))
    for parr in PARROQUIES:
        if parr in textTitol:
            parroquia = parr
            break
    if parroquia == 'NaN':
        textDesc = unidecode.unidecode(oferta.select(params['tag_descripcio'])[0].text.strip().lower().replace("."," ").replace("-"," ").replace(","," "))
        for parr in PARROQUIES:
            if parr in textDesc:
                parroquia = parr                                 
    if parroquia == 'NaN':
        dict_parr = {'1': 'andorra la vella', '2' : 'escaldes', '3' : 'la massana', '4' : 'encamp', '5' : 'canillo', '6' : 'ordino', '9' : 'sant julia'}
        parroquia = dict_parr[str(params['parroquiaActual'])]
    return parroquia
        
        
def getPoble(oferta, params):
    """
    Retorna el poble a partir del tag d'oferta
    :param oferta: bs4.element.tag. Contingut de la oferta  
    :retorna: string indicant el nom del poble      
    """
    poble = 'NaN'
    textTitol = unidecode.unidecode(oferta.select(params['tag_titol'])[0].text.strip().lower().replace("."," ").replace("-"," ").replace(","," "))
    for p in POBLES:
        if p in textTitol:
            poble = p
            break
    if poble == 'NaN':
        textDesc = unidecode.unidecode(oferta.select(params['tag_descripcio'])[0].text.strip().lower().replace("."," ").replace("-"," ").replace(","," "))
        for p in POBLES:
            if p in textDesc:
                poble = p
                break
    if poble == 'NaN':
        poble = getParroquia(oferta, params)
    return poble

def getTipusHabitatge(oferta, params):       
  """
  Retorna el tipus d'habitatge a partir del tag d'oferta
  :param oferta: bs4.element.tag. Contingut de la oferta  
  :retorna: string indicant el tipus d'habitatge (pis or Casa/Xalet/Unifamiliar)        
  """
  tipus_habitatge = ""
  for item in oferta.select(params['tag_descripcio']):
      var = unidecode.unidecode(item.text.strip().lower().replace("."," ").replace("-"," ").replace(","," "))
      if "pis" in var:
          tipus_habitatge = "Pis"
      elif ("xalet" in var) or ("casa" in var) or ("unifamiliar" in var):
          tipus_habitatge = "Casa/Xalet/Unifamiliar"
      else:
          tipus_habitatge = 'NaN'
  return tipus_habitatge

def getDescripcio(oferta, params):      
  """
  Gets the description of the offer
  :param offer: bs4.element.tag. Tag content of the offer
  :return: string. Returns the description of the offer
          
  """
  descripcio = ""
  for item in oferta.select(params['tag_descripcio']):
      descripcio = fix_encoding(item.text.strip().replace('\n', ' ').replace('\r', ''))
  return descripcio.replace(';','')

def getPisos(params):
  l = []
  stop = False
  while not stop:
    url = url_base.format(params['parroquiaActual'], params['pagActual'])
    response = get(url)
    bs = BeautifulSoup(response.text, 'html5lib')
    if bs.select(params['tag_titol']) == params['htmlActual']: 
        params['parroquiaActual'] +=1
        if params['parroquiaActual'] > 2: 
            stop = True
        else: 
            params['pagActual'] = 1
            url = url_base.format(params['parroquiaActual'], params['pagActual'])
            response = get(url)
            bs = BeautifulSoup(response.text, 'html.parser')
    params['htmlActual'] = bs.select(params['tag_titol'])
    params['pagActual'] +=1

    ofertes = bs.select(params['tag_oferta'])
    for oferta in ofertes:
        d={}      
        d['descripcio'] = getDescripcio(oferta, params)
        d['tipusHabitatge'] = getTipusHabitatge(oferta, params)
        d['poble'] = getPoble(oferta, params)
        d['parroquia'] = getParroquia(oferta,params)
        d['dataAnunci'] = getData(oferta)
        d['m2'] = float(getM2(oferta))
        d['numHabitacions'] = float(getNumHabitacions(oferta))
        d['tipusOferta'] = getTipusOferta(oferta)
        d['numVisites'] = float(getNumVisites(oferta))
        d['preu'] = float(getPreu(oferta))
        l.append(d)
    time.sleep(randint(5,20))
  print(len(l))
  return pd.DataFrame(l)  

In [105]:
params = {}
"""Llista de parròquies d'Andorra"""
params['PARROQUIES'] = ["andorra la vella", "escaldes", "la massana", "encamp", "canillo", "ordino", "sant julia"]

"""Llista de pobles d'Andorra"""
params['POBLES'] = ["aldosa de canillo", "el forn", "incles", "meritxell", "els plans","prats","ransol", "soldeu", "el tarter", "el vilar",
                "les bons", "el pas de la casa", "vila", "ansalonga", "arans", "cortinada", "llorts", "segudet", 
                "serrat", "sornas", "aldosa de la massana", "anyos", "arinsal", "erts", "pal", "sispony", "santa coloma", "aixirivall", "aixovall",
                "aubinya", "bixessarri", "certers", "fontaneda", "juberri", "llumeneres", "nagol"]

params['url_base'] = "https://www.buscocasa.ad/ca/filter?ciutat%5B%5D={}&preuMin=0&preuMax=999999999&categ2%5B%5D=17&categ2%5B%5D=10&operacio=1&superficie=&num_habitacions=%3E%3D1&num_banys=%3E%3D1&ref_immo=&pn={}"
params['tag_descripcio'] = 'p.uk-text-break'
params['tag_titol'] = 'div.box-titol'
params['tag_oferta'] = 'div.my-container' 
params['parroquiaActual'] = 1
params['pagActual'] = 1
params['htmlActual'] = ''

df = getPisos(params)


199


In [0]:
#juguem amb les dades
#df.to_csv('patata.csv',sep = '|')
#df['count'] = 1
#df.groupby(['parroquia', 'poble', 'tipusHabitatge']).agg({'preu':'mean', 'm2': 'mean', 'count':'count'}).reset_index().head(20)
#df.sort_values('preu', ascending = True).head(10)
#df.iloc[20]['descripcio']
df.to_csv('patata4.csv',sep = '\t')
#df['descripcio'].head(25)